#### Build a Simple LLM Application with LCEL(Langchain Expression Language)

We will see how to build a simple LLM application with Langchain. This application will translate text from English into another language. This is a relatively simple LLM application-it's just a single LLM call plus some prompting. 

In [20]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [21]:
groq_api_key=os.getenv("GROQ_API_KEY")

In [22]:
from langchain_groq import ChatGroq

In [23]:
model=ChatGroq(model="llama-3.3-70b-versatile",groq_api_key=groq_api_key)
model

ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 32768, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x000002274440C6D0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002274440C9D0>, model_name='llama-3.3-70b-versatile', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [24]:
from langchain_core.messages import HumanMessage,SystemMessage
#helps tell our model which messgae is provided by human and which message is an instruction to the LLM
messages=[
    SystemMessage(content="Translate the following from English to French"),
    HumanMessage(content="Hello how are you?")
]
result=model.invoke(messages)
result

AIMessage(content='Bonjour, comment vas-tu ?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 47, 'total_tokens': 55, 'completion_time': 0.038443312, 'completion_tokens_details': None, 'prompt_time': 0.002440557, 'prompt_tokens_details': None, 'queue_time': 0.056042363, 'total_time': 0.040883869}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_dae98b5ecb', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019c03e7-8d18-7ae0-a49e-565803325ecc-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 47, 'output_tokens': 8, 'total_tokens': 55})

In [25]:
#Now what is instead of this long message I just want the output?
from langchain_core.output_parsers import StrOutputParser
parser=StrOutputParser()
parser.invoke(result)


'Bonjour, comment vas-tu ?'

In [26]:
#Using LCEL I can chain the components
chain=model|parser
chain.invoke(messages)

'Bonjour, comment allez-vous ?'

In [27]:
### Prompt Templates
from langchain_core.prompts import ChatPromptTemplate
generic_template="Translate the following into {language}:"
prompt=ChatPromptTemplate.from_messages([
    ("system",generic_template),("user","{text}") #this text will be entered using runtime

])


In [28]:
result=prompt.invoke({"language":"French","text":"Hello"})

In [29]:
result.to_messages()

[SystemMessage(content='Translate the following into French:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello', additional_kwargs={}, response_metadata={})]

In [30]:
chain=prompt|model|parser

In [31]:
chain.invoke({"language":"French","text":"Hello"})

'Bonjour'